This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'somekey'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [14]:
# First, import the relevant modules
import requests
import json

In [15]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2018-04-24&end_date=2018-04-24&api_key=somekey')

In [16]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-04-15) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-04-29T22:16:41.504Z',
  'newest_available_date': '2020-04-29',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2018-04-24',
  'end_date': '2018-04-24',
  'data': [['2018-04-24',
    None,
    54.75,
    54.15,
    54.6,
    None,
    59651.0,
    3252235.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

# Question 1

In [93]:
# Collect data
afx_x_2017 = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&api_key=somekey')

# Question 2

In [94]:
# Convert JSON object into Python dict
afx_17 = json.loads(afx_x_2017.text)
type(afx_17)

dict

# Question 3

In [95]:
# Highest and Lowest Opening Prices
opening_price_list = [item[1] for item in afx_17['dataset']['data'] if item[1] != None]
max_open = max(opening_price_list)
min_open = min(opening_price_list)
print("Highest opening price is $%.2f, lowest opening price is %.2f$." % (max_opening, min_opening))

Highest opening price is $53.11, lowest opening price is 34.00$.


# Question 4

In [96]:
# Largest change in a day (based on high and low prices)
daily_change = [(items[2] - items[3]) for items in afx_17['dataset']['data']]
largest_change = max(daily_change)
print("The largest change in a day (based on high and low prices) is %.2f$." % largest_change)

The largest change in a day (based on high and low prices) is 2.81$.


# Question 5

In [97]:
# Largest change between any 2 days (based on closing price)
data = afx_17['dataset']['data']
change_in_closing_price = [abs(data[i][4] - data[i+1][4]) for i in range(len(data) - 1) if (data[i][4] != None)]
print("The largest change in any 2 days (based on closing price) is %.2f$." % max(change_in_closing_price))

The largest change in any 2 days (based on closing price) is 2.56$.


# Question 6

In [98]:
# Average daily trading volume
data = afx_17['dataset']['data']
traded_volume = [data[i][6] for i in range(len(data)) if (data[i][6] != None)]
average = sum(traded_volume)/len(traded_volume)
print("The average traded volume is %.2f." % average)

The average traded volume is 89124.34.


# Question 7

In [100]:
# Median traded volume
traded_vol = sorted([items[6] for items in afx_17['dataset']['data']])
traded_len = len(traded_vol)
half_len = traded_len/2
median_1 = traded_vol[round(half_len)]
median_2 = (traded_vol[int(half_len)]+traded_vol[int(half_len)+1])/2
if traded_len % 2 == 1:
    print("The median traded volume is %d." % median_1)
else:
    print("The median traded volume is %d." % median_2)

The median traded volume is 76600.
